In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import tensorflow as tf

In [50]:
from keras.models import load_model

In [64]:
model1 = load_model('my_model.h5')
X=pd.read_csv("ghi.csv")
n = len(X)
X_sa= X[int(n*0.9):]
x_test=X_sa.drop('sm',axis=1)

In [65]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=x_test,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
#     self.val_df = val_df
#     self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [66]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [67]:
wind=WindowGenerator(input_width=24, label_width=24, shift=1)
wind

Total window size: 25
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
Label column name(s): None

In [68]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=24,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [69]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [70]:
# X_1=wind.train
date_time = pd.to_datetime(x_test.pop('ttime'),format='%Y-%m-%d %H:%M')
timestamp_s = date_time.map(pd.Timestamp.timestamp)


In [71]:
day = 24*60*60
year = (365.2425)*day

x_test['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
x_test['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
x_test['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
x_test['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
# time=time_stamp.map(pd.Timestamp.timestamp)
y_pred = model1.predict(wind.train)
# X_1

164/164 [==============================] - 0s 2ms/step


In [73]:
y_pred.shape

(3927, 24, 1)

In [ ]:
model1